In [1]:
from tensorflow import keras 
from tensorflow.keras import layers
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
import pickle
import pandas as pd

epochs = 10
dotrain = True
dotest = True

In [2]:
Y = pd.DataFrame()


prefix = '/home/gomes/Dropbox/gabriel/work/teaching/mine/emeritus/week23_NN_II/code/traffic/2018/flow_station_1209204_2018_'
for i in range(1,12):
    filename = '{}{:02d}'.format(prefix,i)
    with open(filename,'rb') as f:
        X = pickle.load(f)
        X.set_index(pd.to_datetime(X['Timestamp']),inplace=True)
        X.drop(columns=['Timestamp'],inplace=True)
        Y = Y.append(X)
        
prefix = '/home/gomes/Dropbox/gabriel/work/teaching/mine/emeritus/week23_NN_II/code/traffic/2019/flow_station_1209204_2019_'
for i in range(1,12):
    filename = '{}{:02d}'.format(prefix,i)
    with open(filename,'rb') as f:
        X = pickle.load(f)
        X.set_index(pd.to_datetime(X['Timestamp']),inplace=True)
        X.drop(columns=['Timestamp'],inplace=True)
        Y = Y.append(X)
        
        


In [3]:
raw_data = Y['Total Flow'].values
num_train_samples = int(0.5 * len(raw_data))
num_val_samples = int(0.25 * len(raw_data))
num_test_samples = len(raw_data) - num_train_samples - num_val_samples
print("num_train_samples:", num_train_samples)
print("num_val_samples:", num_val_samples)
print("num_test_samples:", num_test_samples)

num_train_samples: 6990
num_val_samples: 3495
num_test_samples: 3495


In [4]:
raw_data.shape

(13980,)

In [5]:
sampling_rate = 1
sequence_length = 24*7 
delay = sequence_length
batch_size = 256
  
train_dataset = keras.preprocessing.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=raw_data[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=False,
    batch_size=batch_size,
    start_index=0,
    end_index=num_train_samples)
  
val_dataset = keras.preprocessing.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=raw_data[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=False,
    batch_size=batch_size,
    start_index=num_train_samples,
    end_index=num_train_samples + num_val_samples)
  
test_dataset = keras.preprocessing.timeseries_dataset_from_array(
    raw_data[:-delay],
    targets=raw_data[delay:],
    sampling_rate=sampling_rate,
    sequence_length=sequence_length,
    shuffle=False,
    batch_size=batch_size,
    start_index=num_train_samples + num_val_samples)

2021-11-26 21:50:57.869032: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
for samples, targets in train_dataset:
    print("samples shape:", samples.shape)
    print("targets shape:", targets.shape)
    print(samples, targets)
    break

samples shape: (256, 168)
targets shape: (256,)
tf.Tensor(
[[ 3637.  3955.  3464. ...  5876.  4618.  3402.]
 [ 3955.  3464.  2810. ...  4618.  3402.  2821.]
 [ 3464.  2810.  2659. ...  3402.  2821.  2216.]
 ...
 [10108. 10658. 10813. ...  9214.  9591. 10094.]
 [10658. 10813. 10804. ...  9591. 10094. 10079.]
 [10813. 10804.  9810. ... 10094. 10079. 10585.]], shape=(256, 168), dtype=float64) tf.Tensor(
[ 2821.  2216.  1954.  2158.  2967.  4828.  7552.  9820.  9424.  9595.
  8641.  8940.  9086.  9173.  9855. 10251.  9974.  9015.  9045.  8030.
  6390.  5958.  5375.  4298.  3510.  2705.  2463.  2065.  2643.  4362.
  7268.  9608.  9833.  8945.  8597.  9178.  9145.  9471.  9411.  9405.
 10018.  9821.  8785.  7544.  5592.  5541.  4314.  3072.  2090.  1466.
  1510.  1313.  2429.  4509.  7682. 10091. 10267.  9668.  9258.  9780.
  9817. 10259. 10937. 11364. 11186. 10151.  8779.  8630.  6769.  5734.
  4885.  3446.  2288.  1643.  1881.  1335.  2306.  4569.  7473. 10275.
 10176. 10154.  9802.  9813.

2021-11-26 21:50:59.643134: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-26 21:50:59.661655: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1800000000 Hz


In [7]:
mean = raw_data[:num_train_samples].mean(axis=0)
raw_data -= mean
std = raw_data[:num_train_samples].std(axis=0)
raw_data /= std

In [8]:
def evaluate_naive_method(dataset):
    total_abs_err = 0. 
    samples_seen = 0 
    for samples, targets in dataset:
        preds = samples[:, -1] * std + mean
        total_abs_err += np.sum(np.abs(preds - targets))
        samples_seen += samples.shape[0]
    return total_abs_err / samples_seen
  
print(f"Validation MAE: {evaluate_naive_method(val_dataset):.2f}") 
print(f"Test MAE: {evaluate_naive_method(test_dataset):.2f}")

Validation MAE: 20350927.57
Test MAE: 19979499.63


# DENSE

In [9]:
if dotrain:

  model_dense = keras.models.Sequential([ keras.Input(shape=sequence_length),
                                            keras.layers.Flatten() ,
                                            keras.layers.Dense(16, activation="relu"),
                                            keras.layers.Dense(1) ])

#   callbacks = [ keras.callbacks.ModelCheckpoint("drive/MyDrive/Colab Notebooks/jena_dense.keras",
#                                       save_best_only=True) ] 

  model_dense.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

  history_dense = model_dense.fit(train_dataset,
                      epochs=1,
                      validation_data=val_dataset)

27/27 [==============================] - 1s 20ms/step - loss: 0.7768 - mae: 0.6909 - val_loss: 25785660.0000 - val_mae: 4372.8799


In [10]:
if dotrain:

  model_1Dconv = keras.models.Sequential([ keras.Input(shape=(sequence_length,1)),
                          layers.Conv1D(8, 16, activation="relu"),
#                           layers.MaxPooling1D(2),
#                           layers.Conv1D(8, 12, activation="relu"),
#                           layers.MaxPooling1D(2),
#                           layers.Conv1D(8, 6, activation="relu"),
                          layers.GlobalAveragePooling1D(),
                          layers.Dense(1) ])

    
#   callbacks = [ keras.callbacks.ModelCheckpoint("drive/MyDrive/Colab Notebooks/jena_1Dconv.keras",
#                                       save_best_only=True) ]
  model_1Dconv.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
  history_1Dconv = model_1Dconv.fit(train_dataset,
                      epochs=1,
                      validation_data=val_dataset)

ValueError: in user code:

    /home/gomes/miniconda3/envs/d178/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/gomes/miniconda3/envs/d178/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/gomes/miniconda3/envs/d178/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/gomes/miniconda3/envs/d178/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/gomes/miniconda3/envs/d178/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/gomes/mi

In [ ]:
if dotrain:

  model_simprnn = keras.models.Sequential([ keras.Input(shape=(sequence_length,1)),
                            layers.SimpleRNN(16, return_sequences=False)  ])                 

#   callbacks = [ keras.callbacks.ModelCheckpoint("drive/MyDrive/Colab Notebooks/jena_simprnn.keras",
#                                       save_best_only=True) ] 

  model_simprnn.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

  history_simprnn = model_simprnn.fit(train_dataset,
                      epochs=1,
                      validation_data=val_dataset)

In [ ]:
sequence_length

In [ ]:
if dotrain:

  model_lstm = keras.models.Sequential([ keras.Input(shape=(sequence_length, 1)),
                            layers.LSTM(16), 
                            layers.Dense(1) ])                 
    
#   callbacks = [ keras.callbacks.ModelCheckpoint("drive/MyDrive/Colab Notebooks/jena_lstm.keras",
#                                       save_best_only=True) ] 

  model_lstm.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

  history_lstm = model_lstm.fit(train_dataset,
                      epochs=epochs,
                      validation_data=val_dataset)